In [ ]:
### Sub sampling
#Want to create a polars frame with the data
import polars as pl
import Utils.constants
import importlib

import Utils.constants as const
import Utils.data_generation as data_gen
import Utils.path_optimisation as path_opt
import Utils.visualisation as vis
import Utils.book_keeping as bk
importlib.reload(const) 
importlib.reload(data_gen)
importlib.reload(path_opt) 
importlib.reload(vis)   
importlib.reload(bk)  

from Utils.constants import FOLDER
from Utils.constants import N_LANES, N_SPEEDS, N_TIME_LIMIT, N_DIRECTIONS, N_SPEED_DEVIATION, N_LANE_DEVIATION, N_TIME_LIMIT
from Utils.constants import MAX_COST_VALUE,INDEX_SPEED_MULTIPLIER
from Utils.data_generation import track_data_import, lane_creation, track_thinning
from Utils.data_generation import load_pl_data, load_np_data



In [ ]:
#from Utils.path_optimisation import processing_optimal_trajectory
import importlib
importlib.reload(data_gen) 
opt_data_path,(av_path,acc_path,tc_path) = data_gen.data_generation("Run1_df")

In [ ]:
#get the data
# opt_data_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Run1_df_l11.parquet'
# av_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Data/av_11'
# acc_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Data/acc_11'
# tc_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Data/tc_11'
opt_df = data_gen.load_pl_data(opt_data_path)
acc_arr = data_gen.load_np_data(acc_path)
av_arr = data_gen.load_np_data(av_path)
tc_arr = data_gen.load_np_data(tc_path)

In [ ]:
# F_MAX_ACC = 20000 #8750
# F_MAX_T = 8750
# MASS = 1000 #typical weihgt of a F1 car 1000 kg
# tau_h = 0.1
# tau_v = 1.0
# g = 10
# mu = 1.0

#F_MAX_ACC = 12000 #8750
F_MAX_ACC = 25000
F_X_NEG_SCALE = 0.3
F_X_MAX_SCALE = 0.1
F_Y_MAX_SCALE = 1.0
MASS = 800 #typical weight of a F1 car 1000 kg
tau_h = 0.62
tau_v = 5.4
g = 10
mu = 1.0
path_opt.add_Force_requiremments(MASS, g, tau_h, tau_v, mu, F_MAX_ACC)


In [ ]:
#cost data used later
cost_df = path_opt.make_cost_df(opt_df,MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE)
importlib.reload(path_opt) 
importlib.reload(data_gen) 
importlib.reload(vis) 
importlib.reload(bk) 
costs, atraj_df = path_opt.run_opt(cost_df, acc_arr, av_arr, MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE, 0.0)
#cost_df.select([c for c in cost_df.columns if "Facc" in c])

In [ ]:

rho_range = [0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.0]
#force_range = [25000, 23000, 21000, 19000, 17000, 15000, 13000]
force_range = [25000]

for f_max in force_range: 
    F_MAX_ACC = f_max
    cost_df = path_opt.make_cost_df(opt_df,MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE)
    for rho in rho_range:
        path_opt.run_opt(cost_df, acc_arr, av_arr, MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE, rho)

In [ ]:
def resample_trajectory_by_time(
    df: pl.DataFrame,
    ds: float,
    dt_target: float,
    x_col: str = "tr_x",
    y_col: str = "tr_y",
    speed_col: str = "speed",
) -> pl.DataFrame:
    """
    Convert a (distance-sampled) trajectory into a (time-sampled) trajectory
    using linear interpolation.
    """
    # 1) add time column
    #df = add_time_column(df, ds, speed_col=speed_col)

    # 2) extract as numpy for interpolation
    t = df["time"].to_numpy()
    x = df[x_col].to_numpy()
    y = df[y_col].to_numpy()
    v = df[speed_col].to_numpy()

    # 3) build regular time grid
    t_max = float(t[-1])
    t_grid = np.arange(0.0, t_max + 1e-9, dt_target)  # small epsilon to include last

    # 4) linear interpolation
    x_grid = np.interp(t_grid, t, x)
    y_grid = np.interp(t_grid, t, y)
    v_grid = np.interp(t_grid, t, v)

    # 5) return as a new Polars frame
    df_reg = pl.DataFrame(
        {
            "time": t_grid,
            x_col: x_grid,
            y_col: y_grid,
            speed_col: v_grid,
        }
    )
    return df_reg

In [ ]:
import numpy as np
c_df = atraj_df.select(["tr_x", "tr_y","speed"]).with_columns(atraj_df["times_est"].cum_sum().alias("time"))


In [ ]:
r_df = resample_trajectory_by_time(c_df, 1.0, 3.0)
r_df

In [68]:
import plotly.express as px
import pandas

def make_car_animation(fig, df: pl.DataFrame):
    # Convert to pandas for Plotly
    pdf = (
        df.select(["time", "tr_x", "tr_y", "speed"])
            #.to_pandas(use_pyarrow_extension_array=False)
    )
    fig.add_trace(go.Scatter(
        pdf,
        x="tr_x",
        y="tr_y",
        animation_frame="time",
        animation_group=None,
        color="speed",
        color_continuous_scale="Viridis",
        range_x=[pdf["tr_x"].min() - 5, pdf["tr_x"].max() + 5],
        range_y=[pdf["tr_y"].min() - 5, pdf["tr_y"].max() + 5],
    )
    )

    fig.update_traces(marker=dict(size=8))
    fig.update_layout(
        title="Car motion",
        xaxis=dict(scaleanchor="y", scaleratio=1),
        width=800,
        height=800,
    )

    # Make the animation a bit smoother/faster if you like
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 40  # ms per frame
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 0

    fig
    return fig
import plotly.graph_objects as go
def add_track_background(fig, ot_df):
    def add_line(i):
        fig.add_trace(go.Scatter(
            x = ot_df["x_lane" + str(i)].to_numpy(),
            y = ot_df["y_lane" + str(i)].to_numpy(),
            mode = "lines",
            marker  = dict(size=1, color="lightgray"),
            line = dict(color="white", width=3),
            showlegend=False
            )
        )   
    for i in range(N_LANES):
        add_line(i)

    fig.update_layout(
        width=800,
        height=1000,
        title="Silverstone Track",
        xaxis=dict(scaleanchor="y", scaleratio=1),
        plot_bgcolor="lightgray",
        paper_bgcolor="#303030", 
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    return fig

fig = make_car_animation(c_df)

TypeError: make_car_animation() missing 1 required positional argument: 'df'

In [67]:
fig = go.Figure()
fig1 = add_track_background(fig, atraj_df)
fig2 = make_car_animation(fig1,c_df)
fig2.show()

AttributeError: 'Figure' object has no attribute 'select'

In [ ]:
import pyarrow